# Настройка колонок для отчёта

In [ ]:
import pandas as pd

columns = [
    # "Код",
    "Номенклатура",
    "Остаток",
    "Суммарное количество",
    "мин.партия отгрузки",
    "Порядок мг01_Воронеж",
    "Порядок мг04_Елец",
    "Порядок мг05_Тамбов",
    "Порядок мг06_Брянск",
    "Порядок мг07_Белгород",
    "Порядок мг08_Курск",
    "Порядок мг09_Воронеж",
    "Порядок мг10_Тамбов",
    "Порядок мг11_Серпухов_РЦ_Подольск",
    "Порядок мг12_СтарыйОскол",
    "Порядок мг14_Орел",
    "Порядок мг16_Таганрог",
    "Порядок мг19_Воронеж",
    "Порядок мг20_Обнинск_РЦ_Подольск",
    "Порядок мг21_Клинцы",
    "Порядок мг23_Липецк",
    "Порядок мг24_Белгород",
    "Порядок мг02_Воронеж",
    "Порядок мг03_Воронеж",
    "Порядок мг15_Россошь",
    "Порядок мг17_Шахты",
    "Порядок мг18_Борисоглебск",
    "Порядок мг25_Ставрополь",
    "Порядок мг26_Воронеж",
    "Порядок мг27_Железногорск",
    "Порядок мг28_Курск",
    "Порядок мг29_Орел",
    "Порядок мг30_Воронеж",
    "Порядок мг31_Армавир",
    "Порядок мг32_Белгород",
    "Порядок мг33_Краснодар",
    "Порядок мг36_Долгопрудный_РЦ_Подольск",
    "Порядок мг37_Липецк",
    "Порядок мг38_Обнинск_РЦ_Подольск",
    "Порядок мг39_Лиски",
    "Порядок мг40_Мичуринск",
    "Порядок мг41_Тверь_РЦ_Подольск",
    "Порядок мг46_Воронеж",
    "Порядок мг51_Мытищи_РЦ_Подольск",
    "Порядок мг53_Губкин",
    "Порядок мг34_Москва_РЦ_Подольск",
    "Порядок мг35_Иваново_РЦ_Подольск",
    "Порядок мг42_Н.Новгород_РЦ_Подольск",
    "Порядок мг45_Домодедово_РЦ_Подольск",
    "Порядок мг47_СтарыйОскол 2",
    "Порядок мг48_Ковров_РЦ_Подольск",
    "Порядок мг50_Смоленск_РЦ_Подольск",
    "Порядок мг52_Москва_РЦ_Подольск",
    "Порядок мг54_Воронеж_Волгоградская",
    "Порядок мг43__Рассказовка, ТЦ Сказка _РЦ_Подольск",
    "Порядок мг55_Брянск",
    "Склад",
    "Объем",
    "Объем общий",
]


# Инициализируем настройки

In [ ]:
### Тестовые данные
source_path = r"./test_data/full_cycle/деф по мин 10.03.23.xlsx"
### Путь до файла куда сохранять
result_path = r"./test_data/full_cycle/result.xlsx"

# Загружаем данные

In [ ]:

source_df = pd.read_excel(
    source_path,
    usecols=[
        "Артикул(доп.)",
        "В",
        "Минимальная партия,ед. (В)",
        "Остаток свободный (В)",
        "Заказано (В)",
        "Количество для перемещения",
        "ОТЗ (В) на норму запаса",
        "Остаток свободный (Из)",
    ],
    sheet_name=0,
)
source_df = source_df.rename(columns={"Артикул(доп.)": "Код"})
source_df = source_df.set_index("Код")

# Устанавливаем фильтры и делаем отчет

In [ ]:
filter1 = source_df["Остаток свободный (В)"] < (
    0.5 * source_df["Минимальная партия,ед. (В)"]
)
filter2 = source_df["Заказано (В)"] == 0
filter3 = source_df["Количество для перемещения"] == 0
filter4 = source_df["Минимальная партия,ед. (В)"] != 1
filter5 = source_df["ОТЗ (В) на норму запаса"] != 0
filter6 = source_df["Остаток свободный (Из)"] != 0
filter7 = source_df["Остаток свободный (Из)"] >= source_df["Минимальная партия,ед. (В)"]

filter = filter1 & filter2 & filter3 & filter4 & filter5 & filter6 & filter7
shops = pd.DataFrame()
shops = source_df[filter]
shops = shops.drop(
    columns=[
        "Остаток свободный (В)",
        "Заказано (В)",
        "Количество для перемещения",
        "ОТЗ (В) на норму запаса",
        "Остаток свободный (Из)",
    ]
)
shops = shops.pivot(columns="В", values="Минимальная партия,ед. (В)")
shops = shops.dropna(how="all")

outcome = pd.DataFrame(columns=columns)
outcome[shops.columns] = shops

# Сохранить результаты в файл

In [ ]:
outcome.to_excel(result_path, index=True, header=True)

# Скопировать результат в буфер обмена

In [ ]:

outcome.to_clipboard(excel=True, index=True, header=True)